In [80]:
import pandas as pd
import numpy as np
import re,os,io
pd.set_option('display.max_rows', 500)    
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

NEW FUNCTIONS FOR NEW DATA

In [62]:
from unidecode import unidecode

def clean_player_name(name):
    if not name: return name
    
    # This turns Greek/Cyrillic "M" into Latin "M" safely
    name = unidecode(str(name))

    name = str(name).strip().upper()
    
    # Map of known variations -> The one true name
    name_map = {
        "55 HOLLERL N": "55 HÖLLERL N",      
        "3 OCONNELL A": "3 O'CONNELL A", 
        "6 DELOW. M": "6 DELOW M",     
        "2 WATSON I": "2 WATSON-BOYE I",  
        "3 WORTHY A": "0 WORTHY A",
        "WORTHY A": "0 WORTHY A",
        "WORTH A": "0 WORTHY A",
        "SHERFIELD G": "25 SHERFIELD G",
        "25. SHERFIELD G": "25 SHERFIELD G",
        "FLANIGAN C": "30 FLANIGAN C",
        "O LOFTON K": "0 LOFTON K",
        "24 TIS CHLER B": "24 TISCHLER B",
        "37 BAUMER I":"37 BAUMER J",
        "32 KELLMAN Κ":"32 KELLMAN K",
        "32 KELLMANN K":"32 KELLMAN K",
        "7 HUGHESE":"7 HUGHES E",
        "10 WILLIAMSONS":"10 WILLIAMSON S",
        "11 ERSEKE":"11 ERSEK E",
        "3 MCCLAIN K":"3 McCLAIN K",
        "11 MUSHIDIK":"11 MUSHIDI K",
        "31 BERAN JRR":"31 BERAN JR R",
        "6 VAN SLOOTENL":"6 VAN SLOOTEN L",
        "CROCKETT JR. T":"0 CROCKETT JR. T",
        "3 FOSTER M":"5 FOSTER M",
        "THOMAS K":"0 THOMAS K",
        "1 OS OBOR G":"1 OSOBOR G",
        "13 BRAXTON JRK":"13 BRAXTON JR K",
        "36 KRAUSEK":"36 KRAUSE K",
        "33 WASHINGTONE":"33 WASHINGTON E",
        "9 LANGJ":"9 LANG J",
        "1 BÄHREJ":"1 BÄHRE J",
        "1 BAHREJ":"1 BÄHRE J",
        "1 BAHRE J":"1 BÄHRE J",
        "21 AMINUJ":"21 AMINU J",
        "GARRETT JR. J":"4 GARRETT JR. J",
        "12 LINGEN M":"12 LINBEN M",
        "21":"21 RAPIEQUE E",
        "24 MANNI":"24 MANN J",
        "24 MANNJ":"24 MANN J",
        "9 HOLLERSBA CHER M":"9 HOLLERSBACHER M",
        "1 STEWART C":"2 STEWART JR. C",
        "2 TH ORPE JR. L":"2 THORPE JR. L",
        "2 THORPE JR . L":"2 THORPE JR. L",
        "21 DANIELS IV. D":"21 DANIELS IV D",
        "43 REED E":"43 REED II E",
        "7 TOKOTO J":"7 TOKOTO II J",
        "GREY J":"8 GREY J",
        "8 GREYJ":"8 GREY J",
        "9 FERNERJ":"9 FERNER J",
        "12 BRAUNE":"12 BRAUN E",
        "40 FENEBERGS":"40 FENEBERG S",
        "17 WIMBERG N":"17 WIMBERG J",
        "13 LE DEE J":"13 LEDEE J",
        "91 MINCHEV Y":"91 MINCHEV J",
        "3 McCLAIN K":"3 MCCLAIN K",
        "3 McCLAIN K":"3 MCCLAIN K",
        "33 McCORMACK D":"33 MCCORMACK D",
        "33 McCORMACK D":"33 MCCORMACK D",
        "3 McCLAIN K":"3 MCCLAIN K",
        "0 HORNED":"0 HORNE D",
        "27 EDIGIN IRE": "27 EDIGIN JR E",
        "8 ELLIS R": "8 ELLIS B",
        "22 PLEISS T":"22 PLEIB T",
        "11 LUCIC V":"11 LUCIC V"
        
        
        
        
    }
    
    return name_map.get(name, name)

In [63]:
import pandas as pd
import re

# --- 1. NORMALIZERS (SAFETY) ---
def normalize_name(name):
    """Standardizes player names (removes trailing dots/spaces)."""
    if not isinstance(name, str): return ""
    return name.strip().rstrip(".,;:")

# create home and away score
def parse_score(score_str):
    """Splits '85-93' into (85, 93). Returns (0, 0) if invalid."""
    try:
        if "-" in score_str:
            h, a = score_str.split("-")
            return int(h.strip()), int(a.strip())
    except:
        pass
    return 0, 0

def parse_diff(diff_str):
    """Converts '+5', '-2', '0' into integer. Returns 0 if invalid."""
    try:
        # Remove '+' and spaces, then convert
        clean_diff = diff_str.replace('+', '').strip()
        return int(clean_diff)
    except:
        return 0

# --- 2. PARSERS of substitutions and quarter starters ---
def parse_substitutions(action_text):
    """Parses multiple substitutions from a single action line."""
    if not isinstance(action_text, str): return [], []
    clean_action = action_text.replace('"', '').replace("'", "").strip()
    clean_action = clean_action.replace('Substitute out.','Substitute out,').replace('Substitute out ','Substitute out,')
    # Regex for 'Name... Substitute out' and 'Name... Substitute in'
    out_matches = re.finditer(r'([\w\s\.\'-]+?)[\s.,;:]+Substitute out', clean_action, re.IGNORECASE)
    in_matches  = re.finditer(r'([\w\s\.\'-]+?)[\s.,;:]+Substitute in',  clean_action, re.IGNORECASE)
    
    players_out = [normalize_name(m.group(1)) for m in out_matches]
    players_in  = [normalize_name(m.group(1)) for m in in_matches]
    
    return players_out, players_in

def parse_starters(text):
    """Extracts players from '[STARTERS]: ...'"""
    match = re.search(r'(?i).*starters\s*:\s*(.*)', text)
    if match:
        raw_players = match.group(1).replace(';', ',')
        return {normalize_name(p) for p in raw_players.split(',') if p.strip()}
    return None

# --- 3. MAIN LOOP ---
def process_full_game(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    if "Quarter" in lines[0] or "quarter" in lines[0]:
        lines = lines[1:]

    # STATE VARIABLES
    active_home = set()
    active_away = set()
    
    # Default to "1" or "Q1" if the very first line is missing it
    last_known_quarter = "1" 
    
    cur_home_score = 0
    cur_away_score = 0
    cur_diff = 0
    
    processed_rows = []

    for line in lines:
        line = line.strip()
        if not line: continue
        
        # 1. SPLIT
        parts = [p.strip() for p in line.split('|')]
        
        # --- 2. ROBUST ALIGNMENT (The Fix for missing quarter values) ---
        
        # A. Remove leading empty column (caused by starting with "|")
        if parts and parts[0] == '':
            parts.pop(0)

        # B. Check Alignment based on Colon (:)
        if parts:
            first_token = parts[0]
            
            # RULE: If it contains a colon, it IS a timestamp, so Quarter is missing.
            if ':' in first_token:
                parts.insert(0, last_known_quarter)
            else:
                # It does NOT have a colon, so it must be the Quarter.
                # Update memory, but assume max length 3 to avoid capturing names like "IVANOVIC"
                # if the file is truly broken.
                if len(first_token) <= 3:
                    last_known_quarter = first_token

        # -------------------------------------

        # 3. PAD COLUMNS
        if len(parts) < 6: 
            parts += [''] * (6 - len(parts))

        # 4. STANDARD ASSIGNMENT
        quarter = parts[0]
        time_str = parts[1]
        home_action = parts[2]
        raw_score = parts[3]
        raw_diff = parts[4]
        away_action = parts[5]


        # --- 1. STARTER RESET ---
        if "Starters" in home_action or "STARTERS" in home_action:
            new = parse_starters(home_action)
            if new: 
                # CLEAN NAMES HERE BEFORE ASSIGNING
                new = {clean_player_name(p) for p in new}
                active_home = new.copy()
        
        if "Starters" in away_action or "STARTERS" in away_action:
            new = parse_starters(away_action)
            if new: 
                # CLEAN NAMES HERE BEFORE ASSIGNING
                new = {clean_player_name(p) for p in new}
                active_away = new.copy()

        # --- 2. SUBSTITUTIONS ---
        if "Substitute" in home_action:
            outs, ins = parse_substitutions(home_action)
            
            # CLEAN NAMES HERE BEFORE PROCESSING
            outs = [clean_player_name(p) for p in outs]
            ins  = [clean_player_name(p) for p in ins]

            for p in outs: active_home.discard(p)
            for p in ins:  active_home.add(p)

        if "Substitute" in away_action:
            outs, ins = parse_substitutions(away_action)
            
            # CLEAN NAMES HERE BEFORE PROCESSING
            outs = [clean_player_name(p) for p in outs]
            ins  = [clean_player_name(p) for p in ins]

            for p in outs: active_away.discard(p)
            for p in ins:  active_away.add(p)

        # --- 3. SCORE & DIFF UPDATE ---
        # Only update if the row actually has data (otherwise keep previous int)
        if raw_score and raw_score.lower() != 'nan':
            cur_home_score, cur_away_score = parse_score(raw_score)
            
        if raw_diff and raw_diff.lower() != 'nan':
            cur_diff = parse_diff(raw_diff)

        # --- 4. EXPAND COLUMNS ---
        home_list = sorted(list(active_home))
        away_list = sorted(list(active_away))
        home_list += [None] * max(0, 5 - len(home_list))
        away_list += [None] * max(0, 5 - len(away_list))

        row_data = {
            'Quarter': quarter,
            'Time': time_str,
            'Home_Action': home_action,
            'Away_Action': away_action,
            # NEW: Integer Columns
            'Home_Score': cur_home_score,
            'Away_Score': cur_away_score,
            'Diff': cur_diff,
            # Lineups
            'Home_1': home_list[0], 'Home_2': home_list[1], 'Home_3': home_list[2], 'Home_4': home_list[3], 'Home_5': home_list[4],
            'Away_1': away_list[0], 'Away_2': away_list[1], 'Away_3': away_list[2], 'Away_4': away_list[3], 'Away_5': away_list[4]
        }
        
        processed_rows.append(row_data)

    return pd.DataFrame(processed_rows)

In [1]:
# TAG EVENT DATA

def tag_complete_events(df):    
    # 1. SHOOTING
    #Taking into account different possible readouts like 2Pt fg, 3 PT FG, 2Ptfg etc...
    
    # 3-Pointers
    df['Home_3P_Make'] = (
                            df['Home_Action'].str.contains(r"3\s*Pt[s]?\.?\s*FG", case=False, na=False) & 
                            df['Home_Action'].str.contains("made", case=False, na=False)
                        ).astype(int) 
    df['Away_3P_Make'] = (
                            df['Away_Action'].str.contains(r"3\s*Pt[s]?\.?\s*FG", case=False, na=False) & 
                            df['Away_Action'].str.contains("made", case=False, na=False)
                        ).astype(int) 

    df['Home_3P_Miss'] = (
                            df['Home_Action'].str.contains(r"3\s*Pt[s]?\.?\s*FG", case=False, na=False) & 
                            (df['Home_Action'].str.contains("miss", case=False, na=False)|
                            df['Home_Action'].str.contains("block", case=False, na=False))
                        ).astype(int)

    df['Away_3P_Miss'] = (
                            df['Away_Action'].str.contains(r"3\s*Pt[s]?\.?\s*FG", case=False, na=False) & 
                            (df['Away_Action'].str.contains("miss", case=False, na=False)|
                            df['Away_Action'].str.contains("block", case=False, na=False))
                        ).astype(int)
    
    # Free Throws
    df['Home_FT_Make'] = (
                            df['Home_Action'].str.contains("Free throw", case=False, na=False) & 
                            df['Home_Action'].str.contains("made", case=False, na=False)
                        ).astype(int) 
                            
    df['Away_FT_Make'] = (
                            df['Away_Action'].str.contains("Free throw", case=False, na=False) & 
                            df['Away_Action'].str.contains("made", case=False, na=False)
                        ).astype(int) 
    
    df['Home_FT_Miss'] = (
                            df['Home_Action'].str.contains("Free throw", case=False, na=False) & 
                            df['Home_Action'].str.contains("miss", case=False, na=False)
                        ).astype(int) 
                            
    df['Away_FT_Miss'] = (
                            df['Away_Action'].str.contains("Free throw", case=False, na=False) & 
                            df['Away_Action'].str.contains("miss", case=False, na=False)
                        ).astype(int) 
    
    #2pt fg
    df['Home_2P_Make'] = (
                            df['Home_Action'].str.contains(r"2\s*Pt[s]?\.?\s*FG", case=False, na=False) & 
                            df['Home_Action'].str.contains("made", case=False, na=False)
                        ).astype(int) 
        
    df['Home_2P_Miss'] = (
                            df['Home_Action'].str.contains(r"2\s*Pt[s]?\.?\s*FG", case=False, na=False) & 
                            (df['Home_Action'].str.contains("miss", case=False, na=False)|
                            df['Home_Action'].str.contains("block", case=False, na=False))
                        ).astype(int)
    
    df['Away_2P_Make'] = (
                            df['Away_Action'].str.contains(r"2\s*Pt[s]?\.?\s*FG", case=False, na=False) & 
                            df['Away_Action'].str.contains("made", case=False, na=False)
                        ).astype(int) 
    
    df['Away_2P_Miss'] = (
                            df['Away_Action'].str.contains(r"2\s*Pt[s]?\.?\s*FG", case=False, na=False) & 
                            (df['Away_Action'].str.contains("miss", case=False, na=False)|
                            df['Away_Action'].str.contains("block", case=False, na=False))
                        ).astype(int)
    # Paint attempts
    df['Home_Paint_FGA'] = df['Home_Action'].str.contains("Lay-up", case=False, na=False).astype(int)|df['Home_Action'].str.contains("Dunk", case=False, na=False).astype(int)
    df['Away_Paint_FGA'] = df['Away_Action'].str.contains("Lay-up", case=False, na=False).astype(int)|df['Away_Action'].str.contains("Dunk", case=False, na=False).astype(int)

    # Paint makes
    df['Home_Paint_FGM'] = ((df['Home_Action'].str.contains("Lay-up", case=False, na=False)|df['Home_Action'].str.contains("Dunk", case=False, na=False)) & df['Home_Action'].str.contains("made", case=False, na=False)).astype(int)
    df['Away_Paint_FGM'] = ((df['Away_Action'].str.contains("Lay-up", case=False, na=False)|df['Away_Action'].str.contains("Dunk", case=False, na=False)) & df['Away_Action'].str.contains("made", case=False, na=False)).astype(int)


    # Fast breaks
    df['Home_Fastbreak'] = df['Home_Action'].str.contains("fast break", case=False, na=False).astype(int)
    df['Away_Fastbreak'] = df['Away_Action'].str.contains("fast break", case=False, na=False).astype(int)
                                                                                                                                                                  
                                                                                                                                                                           
    # 2. AGGREGATE ATTEMPTS (Attempts = Makes + Misses)
    # This is a convenience column for later calculations
    df['Home_FGA'] = df['Home_2P_Make'] + df['Home_2P_Miss'] + df['Home_3P_Make'] + df['Home_3P_Miss']
    df['Away_FGA'] = df['Away_2P_Make'] + df['Away_2P_Miss'] + df['Away_3P_Make'] + df['Away_3P_Miss']
    df['Home_3PA'] = df['Home_3P_Make'] + df['Home_3P_Miss']
    df['Away_3PA'] = df['Away_3P_Make'] + df['Away_3P_Miss'] 
    df['Home_2PA'] = df['Home_2P_Make'] + df['Home_2P_Miss']
    df['Away_2PA'] = df['Away_2P_Make'] + df['Away_2P_Miss']
    df['Home_FTA'] = df['Home_FT_Make'] + df['Home_FT_Miss']
    df['Away_FTA'] = df['Away_FT_Make'] + df['Away_FT_Miss']

    # 3. PLAYMAKING & DEFENSE
    # Assists
    df['Home_Assist'] = df['Home_Action'].str.contains("Assist", case=False,na=False).astype(int)
    df['Away_Assist'] = df['Away_Action'].str.contains("Assist", case=False,na=False).astype(int)
    
    # Blocks
    df['Home_Block'] =df['Home_Action'].str.contains("Block", case=False,na=False).astype(int)
    df['Away_Block'] = df['Away_Action'].str.contains("Block", case=False,na=False).astype(int)
    
    # Steals
    df['Home_Steal'] = df['Home_Action'].str.contains("Steal", case=False,na=False).astype(int)
    df['Away_Steal'] = df['Away_Action'].str.contains("Steal", case=False,na=False).astype(int)

    # 4. REBOUNDS & TURNOVERS
    df['Home_Def_Reb'] = df['Home_Action'].str.contains("Defensive rebound", case=False,na=False).astype(int)
    df['Away_Def_Reb'] = df['Away_Action'].str.contains("Defensive rebound", case=False,na=False).astype(int)
    df['Home_Off_Reb'] = df['Home_Action'].str.contains("Offensive rebound", case=False,na=False).astype(int)
    df['Away_Off_Reb'] = df['Away_Action'].str.contains("Offensive rebound", case=False,na=False).astype(int)
    
    df['Home_Turnover'] = df['Home_Action'].str.contains("Turnover", case=False,na=False).astype(int)
    df['Away_Turnover'] = df['Away_Action'].str.contains("Turnover", case=False,na=False).astype(int)

    # 5. FOULS
    # Useful for calculating "Bonus" situations or opponent FT rate
    df['Home_Foul'] = df['Home_Action'].str.contains("Personal foul", case=False,na=False).astype(int)| df['Home_Action'].str.contains("Unsportsmanlike foul", case=False, na=False).astype(int)
    df['Away_Foul'] = df['Away_Action'].str.contains("Personal foul", case=False,na=False).astype(int)| df['Away_Action'].str.contains("Unsportsmanlike foul", case=False, na=False).astype(int)

    return df

In [2]:
# stint length, also enables us to catch missing or wrong values by duration values that are noticably larger than 20 seconds

def time_str_to_seconds(t_str):
    """Converts 'MM:SS' string to integer seconds."""
    if pd.isna(t_str): return 0
    try:
        parts = str(t_str).split(':')
        minutes = int(parts[0])
        seconds = int(parts[1])
        return minutes * 60 + seconds
    except:
        return 0

def add_duration_and_game_clock(df):
    # 1. Convert Time Column to numeric Seconds immediately
    # This makes math much easier
    df['Seconds_Remaining'] = df['Time'].apply(time_str_to_seconds)
    
    durations = []
    total_elapsed = []
    
    # State Variables
    current_q = None
    prev_seconds_remaining = 0
    
    # Variable to track total time passed in previous quarters (for global game clock)
    cumulative_quarter_time = 0
    
    for index, row in df.iterrows():
        q = row['Quarter']
        curr_seconds = row['Seconds_Remaining']
        
        # --- NEW QUARTER HANDLING ---
        if q != current_q:
            # Add previous quarter's full duration to cumulative tracker
            # (Standard Q is 600s, OT is 300s)
            if current_q is not None:
                # We assume standard quarters are 10 min, OTs are 5 min
                if "OT" in str(current_q):
                    cumulative_quarter_time += 300
                else:
                    cumulative_quarter_time += 600
            
            current_q = q
            
            # Reset the "Previous Time" anchor for the new quarter
            # Most leagues: Q1-Q4 = 10:00 (600s), OT = 5:00 (300s)
            if "OT" or "5" or "6" in str(q):
                prev_seconds_remaining = 300
            else:
                prev_seconds_remaining = 600
        
        # --- DURATION CALCULATION ---
        # Since clock counts DOWN: Duration = Previous - Current
        duration = prev_seconds_remaining - curr_seconds
        
        # Safety: Negative duration (e.g. data error where time jumps up) -> 0
        if duration < 0:
            duration = 0
            
        durations.append(duration)
        
        # --- GLOBAL GAME CLOCK ---
        # (Total Seconds passed since opening tip)
        # = (Time already played in past quarters) + (Time played in this quarter)
        quarter_start_time = 300 if "OT" in str(q) else 600
        time_played_in_q = quarter_start_time - curr_seconds
        total_elapsed.append(cumulative_quarter_time + time_played_in_q)
        
        # Update state for next row
        prev_seconds_remaining = curr_seconds

    # Assign columns
    df['Duration_Seconds'] = durations
    df['Game_Seconds'] = total_elapsed # Good for sorting the whole game chronologically
    
    return df

In [ ]:
#gamelist = list(range(2003986,2004112))

gamelist = [2003986,2003987]

for gamenr in gamelist:
    inputfile = f'data/{gamenr}.txt'
    
    if not os.path.exists(inputfile):
            print(f"Skipping {gamenr}: File not found.")
            continue

    df = process_full_game(inputfile)
    
    df = add_duration_and_game_clock(df)
    
    df = tag_complete_events(df)
    
    df['gameid'] = gamenr
    
    df.to_csv(f'data/{gamenr}eval.txt',index = False)